# Load Library

In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import plotly.express as px
import os

# Connect To BigQuery

#### To Get JSON APi File follow this instructions
<https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas>

In [2]:
# Setup Google application Credentials environment 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'ace-fiber-325220-6e2a18fd2882.json' ## Note : You will need your personel JSON APi 

# Construct a BigQuery client object
Client = bigquery.Client()

In [3]:
# Qeury Chicago Crime Data Before Covid-19 Pandemic
Before_Covid = """
    SELECT *
    FROM `bigquery-public-data.chicago_crime.crime`
    WHERE date BETWEEN TIMESTAMP("2018-02-20") AND TIMESTAMP("2020-01-24")
    ORDER BY date
"""

# Qeury Chicago Crime Data After Covid-19 Pandemic
After_Covid = """
    SELECT *
    FROM `bigquery-public-data.chicago_crime.crime`
    WHERE date BETWEEN TIMESTAMP("2020-01-24") AND TIMESTAMP("2021-03-16")
    ORDER BY date 
"""

In [4]:
# Make API Request 
query_job_after_covid = Client.query(After_Covid)
query_job_before_covid = Client.query(Before_Covid)

In [5]:
df_after = query_job_after_covid.result()
df_before = query_job_before_covid.result()

In [6]:
# Transfer Data to DataFrame 
data_after = df_after.to_dataframe() 
data_before = df_before.to_dataframe()

In [7]:
after_copy = data_after.copy()
before_copy = data_before.copy()

# Explore Data

In [8]:
## Read Meta-Data 
Meta_Data = pd.read_csv('Meta-Data.csv', encoding= 'unicode_escape')
Meta_Data

,Column Name,Description,Type
0,ID,Unique identifier for the record.,Number
1,Case Number,The Chicago Police Department RD Number (Recor...,Plain Text
2,Date,Date when the incident occurred. this is somet...,Date & Time
3,Block,The partially redacted address where the incid...,Plain Text
4,IUCR,The Illinois Unifrom Crime Reporting code. Thi...,Plain Text
5,Primary Type,The primary description of the IUCR code.,Plain Text
6,Description,"The secondary description of the IUCR code, a ...",Plain Text
7,Location Description,Description of the location where the incident...,Plain Text
8,Arrest,Indicates whether an arrest was made.,Checkbox
9,Domestic,Indicates whether the incident was domestic-re...,Checkbox


In [9]:
# Check data type & shape of each dataset 
before_copy.info()
after_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512061 entries, 0 to 512060
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype              
---  ------                --------------   -----              
 0   unique_key            512061 non-null  int64              
 1   case_number           512061 non-null  object             
 2   date                  512061 non-null  datetime64[ns, UTC]
 3   block                 512061 non-null  object             
 4   iucr                  512061 non-null  object             
 5   primary_type          512061 non-null  object             
 6   description           512061 non-null  object             
 7   location_description  509870 non-null  object             
 8   arrest                512061 non-null  bool               
 9   domestic              512061 non-null  bool               
 10  beat                  512061 non-null  int64              
 11  district              512061 non-null  int64        

### Check For Duplicates

In [10]:
# Check for duplicates in unique_key  & case_number columns 
# According to MetaData it shouldn't be duplicated value in this column 
before_copy.duplicated('case_number').value_counts()

False    512018
True         43
dtype: int64

In [11]:
after_copy.duplicated('case_number').value_counts()

False    233593
True         32
dtype: int64

In [12]:
before_copy.duplicated('unique_key').value_counts()

False    512061
dtype: int64

In [13]:
after_copy.duplicated('unique_key').value_counts()

False    233625
dtype: int64

### Check For NULL Values

In [14]:
# Check for null value in primary_type column
before_copy.primary_type.isnull().value_counts()

False    512061
Name: primary_type, dtype: int64

In [15]:
# Check for null value in primary_type column
after_copy.primary_type.isnull().value_counts()

False    233625
Name: primary_type, dtype: int64

In [16]:
# Check for null value in location_description column
before_copy.location_description.isnull().value_counts()

False    509870
True       2191
Name: location_description, dtype: int64

In [17]:
# Check for null value in location_description column
after_copy.location_description.isnull().value_counts()

False    232278
True       1347
Name: location_description, dtype: int64

# Clean & Transform Data

### Remove Duplicates

In [18]:
# This code to remove duplicates from case_number column which is unique to the incident
# according to MetaData so it shouldn't be duplicated value 
before_copy = before_copy.drop_duplicates('case_number')
after_copy = after_copy.drop_duplicates('case_number')

### Extract Time & Date

In [19]:
# This code to extract day name from date column 
before_copy['Day_Name'] = before_copy['date'].dt.day_name()
after_copy['Day_Name'] = after_copy['date'].dt.day_name()

In [20]:
# This code to extract day from date column
before_copy['Day'] = before_copy['date'].dt.day
after_copy['Day'] = after_copy['date'].dt.day

In [21]:
# This code to extract month from date column 
before_copy['Month'] = before_copy['date'].dt.month
after_copy['Month'] = after_copy['date'].dt.month

In [22]:
# This code to extract year from date column 
before_copy['Year'] = before_copy['date'].dt.year
after_copy['Year'] = after_copy['date'].dt.year

In [23]:
# This code to extract time from date column & change format to PM/AM 
before_copy['Time'] = before_copy['date'].dt.time.apply(lambda x: x.strftime('%H %p'))
after_copy['Time'] = after_copy['date'].dt.time.apply(lambda x: x.strftime('%H %p'))

### Change False & True To No & Yes

In [24]:
# Change False & True into No & Yes in arrest column, That change type of column from boolean into string (object)
before_copy['arrest'] = before_copy.arrest.replace({True: 'Yes', False: 'No'})
after_copy['arrest'] = after_copy.arrest.replace({True: 'Yes', False: 'No'})

In [25]:
# Change False & True into No & Yes in domestic column, That change type of column from boolean into string (object)
before_copy['domestic'] = before_copy.domestic.replace({True: 'Yes', False: 'No'})
after_copy['domestic'] = after_copy.domestic.replace({True: 'Yes', False: 'No'})

### Drop No Need Columns

In [26]:
# Drop columns no need for analysis 
before_copy = before_copy.drop(['case_number', 'description', 'district', 'beat', 'x_coordinate', 'y_coordinate', 'block', 'iucr', 'year', 'community_area', 'fbi_code', 'ward', 'updated_on'], axis=1)

In [27]:
# Drop columns no need for analysis 
after_copy = after_copy.drop(['case_number', 'description', 'district', 'beat', 'x_coordinate', 'y_coordinate', 'block', 'iucr', 'year', 'community_area', 'fbi_code', 'ward', 'updated_on'], axis=1)

# Descriptive analysis With Visualization

### Compare number of crime before & after pendemic by crime type 

In [28]:
# count number of crime by type after pendemic
after_primary_type_groupby = after_copy.groupby(['primary_type'])[['unique_key']].count().nlargest(10, 'unique_key')

# count number of crime by type before pendemic
before_primary_type_groupby = before_copy.groupby(['primary_type'])[['unique_key']].count().nlargest(10, 'unique_key')

# merge two results after & before into one table 
before_after_primary_type = pd.concat([before_primary_type_groupby, after_primary_type_groupby],axis=1,ignore_index=False)

# rename columns 
before_after_primary_type.columns=['Before Pendemic', 'After Pendemic']

# Calculate the percent of change on primary type 
before_after_primary_type['% Of Change'] = ((before_after_primary_type['After Pendemic']-before_after_primary_type['Before Pendemic'])/before_after_primary_type['Before Pendemic']*100)

# show the table
before_after_primary_type.reset_index()

,primary_type,Before Pendemic,After Pendemic,% Of Change
0,THEFT,123726.0,43995.0,-64.441589
1,BATTERY,96311.0,45527.0,-52.729179
2,CRIMINAL DAMAGE,52832.0,27633.0,-47.696472
3,ASSAULT,39854.0,20403.0,-48.805641
4,DECEPTIVE PRACTICE,36985.0,21661.0,-41.433013
5,OTHER OFFENSE,32876.0,14078.0,-57.178489
6,NARCOTICS,27863.0,NaN,NaN
7,BURGLARY,20504.0,9224.0,-55.013656
8,MOTOR VEHICLE THEFT,18019.0,11248.0,-37.577002
9,ROBBERY,16856.0,8786.0,-47.876127


In [29]:
Primary_Type_Viz = px.bar(before_after_primary_type,
                    barmode='group',
                    height=800,
                    width=800,
                    title="Primary Type Before And After Covid",                
)
Primary_Type_Viz.show()

### Compare number of crime before & after pendemic by location description

In [30]:
# Count number of crime by location description after pendemic (Return  top 10 largest only)
after_location_description_groupby = after_copy.groupby(['location_description'])[['unique_key']].count().nlargest(10, 'unique_key')

# Count number of crime by location description before pendemic (Return  top 10 largest only)
before_location_description_groupby = before_copy.groupby(['location_description'])[['unique_key']].count().nlargest(10, 'unique_key')

# merge two results after & before into one table 
before_after_location_description = pd.concat([before_location_description_groupby, after_location_description_groupby],axis=1,ignore_index=True)

# rename columns 
before_after_location_description.columns=['Before Pendemic', 'After Pendemic']

# Calculate the percent of change on location description 
before_after_location_description['% Of Change'] = ((before_after_location_description['After Pendemic']-before_after_location_description['Before Pendemic'])/before_after_location_description['Before Pendemic']*100)

# Show table 
before_after_location_description.reset_index()

,location_description,Before Pendemic,After Pendemic,% Of Change
0,STREET,111724.0,56256.0,-49.647345
1,RESIDENCE,85164.0,43392.0,-49.048894
2,APARTMENT,67504.0,42344.0,-37.271865
3,SIDEWALK,40584.0,14021.0,-65.451902
4,OTHER,20545.0,NaN,NaN
5,PARKING LOT/GARAGE(NON.RESID.),14529.0,NaN,NaN
6,SMALL RETAIL STORE,13746.0,5634.0,-59.013531
7,RESTAURANT,13688.0,NaN,NaN
8,ALLEY,9911.0,5204.0,-47.492685
9,VEHICLE NON-COMMERCIAL,9644.0,NaN,NaN


In [31]:

Location_Description_Viz = px.bar(before_after_location_description,
                        height = 920,
                        width = 720,
                        barmode='group',
                        title="Location Description Before And After Covid",
)

Location_Description_Viz.show()

### Compare number of arrest & domestic before & after pendemic

##### Arrest

In [32]:
# Count number of arrest made after pendemic
after_arrest_groupby = after_copy.groupby(['arrest'])[['unique_key']].count()

# Count number of arrest made before pendemic
before_arrest_groupby = before_copy.groupby(['arrest'])[['unique_key']].count()

# merge two results after & before into one table 
before_after_arrest = pd.concat([before_arrest_groupby, after_arrest_groupby], axis=1, ignore_index=False)

# rename columns 
before_after_arrest.columns=['Arrest Before Pendemic', 'Arrest After Pendemic']

# Calculate the percent of change arrest 
before_after_arrest['% Of Change'] = ((before_after_arrest['Arrest After Pendemic']-before_after_arrest['Arrest Before Pendemic'])/before_after_arrest['Arrest Before Pendemic'] *100)

# Show table 
before_after_arrest.reset_index()

,arrest,Arrest Before Pendemic,Arrest After Pendemic,% Of Change
0,No,405595,197588,-51.284409
1,Yes,106423,36005,-66.168028


In [33]:
Arrest_Viz = px.bar(before_after_arrest, title="Arrest Before And After Covid",
                        height=920,
                        width=720,
                        barmode='group',
)
Arrest_Viz.show()

#### Domestic

In [34]:
# Count number of domestic after pendemic
after_domestic_groupby = after_copy.groupby(['domestic'])[['unique_key']].count()

# Count number of domestic before pendemic
before_domestic_groupby = before_copy.groupby(['domestic'])[['unique_key']].count()

# merge two results after & before into one table 
before_after_domestic = pd.concat([before_domestic_groupby, after_domestic_groupby], axis=1, ignore_index=False)

# rename columns 
before_after_domestic.columns=['Number Of Domestic Before Pendemic', 'Number Of Domestic After Pendemic']

# Calculate the percent of domestic change 
before_after_domestic['% Of Change'] = ((before_after_domestic['Number Of Domestic After Pendemic']-
                                                                        before_after_domestic['Number Of Domestic Before Pendemic'])
                                                                        /before_after_domestic['Number Of Domestic Before Pendemic'] *100)

# Show table 
before_after_domestic.reset_index()

,domestic,Number Of Domestic Before Pendemic,Number Of Domestic After Pendemic,% Of Change
0,No,427509,188236,-55.969114
1,Yes,84509,45357,-46.328793


In [35]:
Arrest_Viz = px.bar(before_after_domestic, title="Domestic Before And After Covid",
                        height=920,
                        width=720,
                        barmode='group',
)
Arrest_Viz.show()